# DynamoDBのテーブルから項目数を取得する

In [ ]:
import boto3
from tqdm.notebook import tqdm

## AWS config設定

In [ ]:
# ローカルPC上で `localstack` に接続する設定にしています
aws_config = {
   "aws_access_key_id": "dummy",
   "aws_secret_access_key": "dummy",
   "region_name": "ap-northeast-1",
   "endpoint_url": "http://localstack:4566"
}

## client インスタンスを生成

In [ ]:
client = boto3.client(service_name="dynamodb", **aws_config)
dynamodb = boto3.resource(service_name="dynamodb", **aws_config)

## テーブルリスト取得

In [ ]:
table_list = client.list_tables()
table_list['TableNames']

['LocalTest']

## テーブルの件数取得

In [ ]:
# `LocalTest` テーブルの件数取得
table = dynamodb.Table("LocalTest")

### DynamoDB.Table.item_count

In [ ]:
# `item_count` は6時間ごとに更新された値が取得されます（ローカルの場合リアルタイムで取得できます）
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table.item_count
table.item_count

100000

### DynamoDB.Table.scan

In [ ]:
# `scan` で一度に取得できるデータサイズは最大1MBに制限されています。
# 条件なしで `COUNT` を取得しても1MBまでの件数しか取得できません。
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table.scan
ret = table.scan(Select='COUNT')
ret['Count']

16298

### DynamoDB.Table.scan with Pagenator

In [ ]:
# `item_count` および `scan` の制限を回避して全項目を取得する方法として、 `scan` の説明にあるようにページ分割をして取得する方法があります。
# 以下、全項目数（レコード数）を取得する例を示していますが、 `page["Items"]` で項目も取得できます。
total = 0
paginator = client.get_paginator("scan")
for page in tqdm(paginator.paginate(TableName="LocalTest", PaginationConfig={
        'MaxItems': 500000 # 全項目数より大きな値を設定
    })):
    total += page['Count']
    print(total, end='->')
print(total)

16298->32589->48879->65183->81475->97776->100000->
100000
